<a href="https://colab.research.google.com/github/LaioGomes/Previsao-de-preco-Ethereum/blob/main/EthereumAnalisePreditiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

In [ ]:
hoje = datetime.today().strftime('%Y-%m-%d')
diaInicio = '2016-01-01'

dfEth = yf.download('ETH-USD', diaInicio, hoje)

dfEth.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-06-23,"$1,051.33","$1,146.49","$1,050.22","$1,143.39","$1,143.39",14657862919
2022-06-24,"$1,143.20","$1,238.62","$1,134.51","$1,226.84","$1,226.84",16981552654
2022-06-25,"$1,226.72","$1,246.39","$1,184.56","$1,243.45","$1,243.45",12481946184
2022-06-26,"$1,242.99","$1,272.13","$1,199.41","$1,199.83","$1,199.83",12096607824
2022-06-27,"$1,199.77","$1,234.06","$1,181.19","$1,204.78","$1,204.78",12537337856


In [ ]:
dfEth.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1692 entries, 2017-11-09 to 2022-06-27
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1692 non-null   float64
 1   High       1692 non-null   float64
 2   Low        1692 non-null   float64
 3   Close      1692 non-null   float64
 4   Adj Close  1692 non-null   float64
 5   Volume     1692 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 92.5 KB


In [ ]:
dfEth.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
dfEth.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
dfEth.reset_index(inplace=True)

dfEth.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

####The prophet library requires us to have only two columns in our data frame — “ds” and “y”, which is the dateandopen columns respectively

In [ ]:
df = dfEth[["Date", "Open"]]

newNames = {
    "Date": "ds",
    "Open": "y",
}

df.rename(columns=newNames, inplace=True)
df.tail()

,ds,y
1687,2022-06-23,"$1,051.33"
1688,2022-06-24,"$1,143.20"
1689,2022-06-25,"$1,226.72"
1690,2022-06-26,"$1,242.99"
1691,2022-06-27,"$1,199.77"


In [ ]:
x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

fig.update_layout(
    title_text = "Gráfico temporal do preço de abertura do Ethereum"
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
            
        ),
        rangeslider=dict(visible=True),
        type='date',
    )
)

In [ ]:
m = Prophet(
    seasonality_mode="multiplicative"
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
futuro = m.make_future_dataframe(periods = 365)

futuro.tail()

,ds
2052,2023-06-23
2053,2023-06-24
2054,2023-06-25
2055,2023-06-26
2056,2023-06-27


In [ ]:
previsao = m.predict(futuro)
previsao[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2052,2023-06-23,$425.92,$-573.75,"$1,413.54"
2053,2023-06-24,$414.42,$-582.49,"$1,417.03"
2054,2023-06-25,$409.67,$-551.97,"$1,369.74"
2055,2023-06-26,$404.84,$-562.81,"$1,369.11"
2056,2023-06-27,$397.72,$-564.97,"$1,396.94"


In [ ]:
diaSeguinte = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

previsao[previsao['ds'] == diaSeguinte]['yhat'].item()

1348.8650711676605

In [ ]:
plot_plotly(m, previsao)

In [ ]:
plot_components_plotly(m, previsao)